# Imports

In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import pandas as pd
import helpers.hdatetime as hdateti

# Get trades data by CryptoChassis

In [2]:
import im_v2.crypto_chassis.data.extract.extractor as imvccdexex

contract_type = "futures"

cc_extractor = imvccdexex.CryptoChassisExtractor(
    contract_type
)

In [3]:
currency_pair = "BTC_USDT"
exchange_id = "binance"
start_timestamp = pd.Timestamp("2023-01-01 00:00:01")
end_timestamp = hdateti.convert_timestamp_to_unix_epoch(pd.Timestamp("2023-01-01 00:10:01"), unit="s")


cc_data = cc_extractor._download_trades(
    exchange_id,
    currency_pair,
    start_timestamp=start_timestamp
)

In [4]:
cc_data = cc_data[cc_data.timestamp < end_timestamp]
cc_data

,timestamp,price,size,is_buyer_maker
0,1672531204,16537.5,0.004,1
1,1672531204,16537.6,0.232,0
2,1672531204,16537.5,0.445,1
3,1672531204,16537.6,0.792,0
4,1672531204,16537.5,0.081,1
...,...,...,...,...
2314,1672531799,16521.5,0.013,0
2315,1672531799,16521.5,4.498,0
2316,1672531799,16521.5,0.553,0
2317,1672531800,16521.4,0.605,1


In [5]:
cc_data[["timestamp"]].agg(["min", "max"]).apply(pd.to_datetime, unit="s")

,timestamp
min,2023-01-01 00:00:04
max,2023-01-01 00:10:00


# Get trades data by Ccxt

 Seems like we don't need takeOrMaker
 https://github.com/ccxt/ccxt/issues/4201

In [6]:
import im_v2.ccxt.data.extract.extractor as ivcdexex

exchange_id = "binance"
contract_type = "futures"

ccxt_extractor = ivcdexex.CcxtExtractor(exchange_id, contract_type)

In [7]:
data_type = "trades"
currency_pair = "BTC_USDT"
start_timestamp = pd.Timestamp("2023-01-01 00:00:01")
end_timestamp = pd.Timestamp("2023-01-01 00:10:01")
ccxt_data = ccxt_extractor.download_data(
    data_type,
    exchange_id,
    currency_pair,
    start_timestamp=start_timestamp,
    end_timestamp=end_timestamp
)
ccxt_data

,timestamp,symbol,side,price,amount,end_download_timestamp
0,1672531204118,BTC/USDT:USDT,sell,16537.5,0.004,2023-02-06 17:18:18.159622+00:00
1,1672531204178,BTC/USDT:USDT,buy,16537.6,0.232,2023-02-06 17:18:18.159622+00:00
2,1672531204209,BTC/USDT:USDT,sell,16537.5,0.445,2023-02-06 17:18:18.159622+00:00
3,1672531204229,BTC/USDT:USDT,buy,16537.6,0.792,2023-02-06 17:18:18.159622+00:00
4,1672531204229,BTC/USDT:USDT,sell,16537.5,0.081,2023-02-06 17:18:18.159622+00:00
...,...,...,...,...,...,...
2314,1672531799211,BTC/USDT:USDT,buy,16521.5,0.013,2023-02-06 17:18:18.159622+00:00
2315,1672531799475,BTC/USDT:USDT,buy,16521.5,4.498,2023-02-06 17:18:18.159622+00:00
2316,1672531799917,BTC/USDT:USDT,buy,16521.5,0.553,2023-02-06 17:18:18.159622+00:00
2317,1672531800477,BTC/USDT:USDT,sell,16521.4,0.605,2023-02-06 17:18:18.159622+00:00


In [8]:
ccxt_data[["timestamp"]].agg(["min", "max"]).apply(pd.to_datetime, unit="ms")

,timestamp
min,2023-01-01 00:00:04.118
max,2023-01-01 00:10:00.936


In [9]:
cc_data.rename(columns={"size": "amount"}, inplace=True)

In [10]:
columns_to_compare = ["price", "amount"]
(cc_data[columns_to_compare] == ccxt_data[columns_to_compare]).all()

price     True
amount    True
dtype: bool